In [5]:
# -*- coding: utf-8 -*-
"""
Solver for the given key-check program (QNX target).

Model:
- string_to_code mapping (from user-provided code):
    act=0, con=0, abort=1, cancel=1, enable=2, start=2, stop=3, reset=4, reboot=4
- The checker consumes the first 10 chars as hex-like digits -> d0..d9 in [0..15].
- These digits are placed into a 5x5 matrix at fixed positions; from code we derive 5 row-constraints:
    R0: d0 + d2 + d3 = 34
    R1: d1 + d3 + d6 = 34
    R2: d2 + d4 + d8 = 34
    R3: d4 + d7 + d9 = 34
    R4: d5 + d6 + d7 = 34
- QNX-specific global constraint (devctl/open/stat on QNX succeed, v38[8]&0x80==0):
    8*d0 = 58 + d9   => (d0, d9) ∈ {(8,6), (9,14)}
- Final 4 chars (indices 10..13) must satisfy v50==100, with weights (+45,+67,+8,+9 for 'a'; negated for 's'):
    Unique compact choice: 'a','s','a','s'  -> total change = -23 (123 -> 100)
- Output keys are 14 chars: first 10 from d0..d9 (hex), then "asas".

This script enumerates all solutions under those constraints and prints up to N keys.
"""

from typing import List, Tuple

HEX = "0123456789abcdef"

def int_to_hex_char(x: int) -> str:
    if 0 <= x <= 15:
        return HEX[x]
    raise ValueError("digit out of range (need 0..15)")

def digits_to_prefix(d: List[int]) -> str:
    return "".join(int_to_hex_char(x) for x in d)

def solve_qnx(max_solutions: int = 1000) -> List[str]:
    """Enumerate solutions for the QNX target (recommended/realistic path)."""
    solutions: List[str] = []

    # From 8*d0 = 58 + d9 we only have these two pairs in 0..15 domain
    pairs: List[Tuple[int,int]] = [(8, 6), (9, 14)]

    for d0, d9 in pairs:
        # R0 fixes sum of d2+d3 depending on d0
        s23 = 34 - d0  # (d2 + d3)
        # R3 ties d4 + d7 with d9
        s47 = 34 - d9  # (d4 + d7)

        for d2 in range(16):
            d3 = s23 - d2
            if not (0 <= d3 <= 15): 
                continue

            # enumerate (d4, d7)
            # d4 in [max(0, s47-15) .. min(15, s47)]
            lo4 = max(0, s47 - 15)
            hi4 = min(15, s47)
            for d4 in range(lo4, hi4 + 1):
                d7 = s47 - d4
                if not (0 <= d7 <= 15):
                    continue

                # R2 fixes d8
                d8 = 34 - d2 - d4
                if not (0 <= d8 <= 15):
                    continue

                # Now use R4 and R1 to pick d6, then derive d5 and d1
                # R4: d5 + d6 + d7 = 34  => d5 = 34 - d7 - d6
                # R1: d1 + d3 + d6 = 34  => d1 = 34 - d3 - d6
                for d6 in range(16):
                    d5 = 34 - d7 - d6
                    if not (0 <= d5 <= 15):
                        continue
                    d1 = 34 - d3 - d6
                    if not (0 <= d1 <= 15):
                        continue

                    digits = [d0, d1, d2, d3, d4, d5, d6, d7, d8, d9]
                    prefix = digits_to_prefix(digits)
                    key = prefix + "asas"  # indices 10..13
                    solutions.append(key)

                    if len(solutions) >= max_solutions:
                        return solutions

    return solutions

# (Optional) quick, strict verifier for QNX model
def verify_qnx(key: str) -> bool:
    if len(key) > 14:
        return False
    # parse first 10 as the program does for "hex-like"
    d = []
    for ch in key[:10]:
        if '1' <= ch <= '9':
            d.append(ord(ch) - 48)
        elif ch == '0':
            d.append(0)
        elif 'a' <= ch <= 'f':
            d.append(10 + ord(ch) - 97)
        else:
            d.append(16)  # we don't generate this in solve_qnx(), but mimic program
    if any(x < 0 or x > 15 for x in d):
        return False

    d0,d1,d2,d3,d4,d5,d6,d7,d8,d9 = d

    # Five row-sum constraints
    if not (d0 + d2 + d3 == 34): return False
    if not (d1 + d3 + d6 == 34): return False
    if not (d2 + d4 + d8 == 34): return False
    if not (d4 + d7 + d9 == 34): return False
    if not (d5 + d6 + d7 == 34): return False

    # QNX global equality: v47 = 24 + d9, v24 = 8*d0 (+ 0x00), and v47 == v24 - 34
    if not (24 + d9 == 8 * d0 - 34):
        return False

    # v50 constraint: positions 10..13
    tail = key[10:14]
    if len(tail) != 4:  # need exactly 4 chars for our intended construction
        return False
    v50 = 123
    # idx10 (+45 for 'a', -45 for 's')
    v50 += 45 if tail[0] == 'a' else (-45 if tail[0] == 's' else 0)
    # idx11 (+67 / -67)
    v50 += 67 if tail[1] == 'a' else (-67 if tail[1] == 's' else 0)
    # idx12 (+8 / -8)
    v50 += 8  if tail[2] == 'a' else (-8  if tail[2] == 's' else 0)
    # idx13 (+9 / -9)
    v50 += 9  if tail[3] == 'a' else (-9  if tail[3] == 's' else 0)
    if v50 != 100:
        return False

    return True

if __name__ == "__main__":
    sols = solve_qnx(max_solutions=1000000)  # adjust as you like
    for k in sols:
        print(k, "OK" if verify_qnx(k) else "!!BAD")
    print(f"总共Key数: {len(sols)}")


8fbfdf4fa6asas OK
8ebfde5fa6asas OK
8dbfdd6fa6asas OK
8cbfdc7fa6asas OK
8bbfdb8fa6asas OK
8abfda9fa6asas OK
89bfd9afa6asas OK
88bfd8bfa6asas OK
87bfd7cfa6asas OK
86bfd6dfa6asas OK
85bfd5efa6asas OK
84bfd4ffa6asas OK
8ebfef5e96asas OK
8dbfee6e96asas OK
8cbfed7e96asas OK
8bbfec8e96asas OK
8abfeb9e96asas OK
89bfeaae96asas OK
88bfe9be96asas OK
87bfe8ce96asas OK
86bfe7de96asas OK
85bfe6ee96asas OK
84bfe5fe96asas OK
8dbfff6d86asas OK
8cbffe7d86asas OK
8bbffd8d86asas OK
8abffc9d86asas OK
89bffbad86asas OK
88bffabd86asas OK
87bff9cd86asas OK
86bff8dd86asas OK
85bff7ed86asas OK
84bff6fd86asas OK
8fcede5f96asas OK
8ecedd6f96asas OK
8dcedc7f96asas OK
8ccedb8f96asas OK
8bceda9f96asas OK
8aced9af96asas OK
89ced8bf96asas OK
88ced7cf96asas OK
87ced6df96asas OK
86ced5ef96asas OK
85ced4ff96asas OK
8fceef5e86asas OK
8eceee6e86asas OK
8dceed7e86asas OK
8cceec8e86asas OK
8bceeb9e86asas OK
8aceeaae86asas OK
89cee9be86asas OK
88cee8ce86asas OK
87cee7de86asas OK
86cee6ee86asas OK
85cee5fe86asas OK
8eceff6d76